<center><a href="https://www.nvidia.com/dli"> <img src="https://github.com/sathyasheelans/NVIDIA_DL/blob/master/Nvidia%20-%20Intro%20to%20Deep%20Learning/Lesson_1/images/DLI_Header.png?raw=1" alt="Header" style="width: 400px;"/> </a></center>

# 1. Image Classification with the MNIST Dataset

Create a model in both pytorch and and tensorflow and compare the differences

## 1.1 Objectives

### Install pytorch and tensorflow

In [30]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True


In [28]:
!pip install triton --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.1 MB/s eta 0:00:00


In [1]:
!pip3 install torch torchvision torchaudio

### Load Pytorch

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# Visualization tools
import torchvision
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In [5]:
train_set = torchvision.datasets.MNIST("./data/", train=True, download=True)
valid_set = torchvision.datasets.MNIST("./data/", train=False, download=True)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9912422/9912422 [00:02<00:00, 4652883.09it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28881/28881 [00:00<00:00, 134401.83it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1648877/1648877 [00:01<00:00, 1263317.49it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4542/4542 [00:00<00:00, 4925162.56it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



We stated above that the MNIST dataset contained 70,000 grayscale images of handwritten digits. By executing the following cells, we can see that Keras has partitioned 60,000 of these [PIL Images](https://pillow.readthedocs.io/en/stable/reference/Image.html) for training, and 10,000 for validation (after training).

## 1.4 Preparing the Data for Training

In [6]:
trans = transforms.Compose([transforms.ToTensor()])

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [7]:
train_set.transform = trans
valid_set.transform = trans

In [8]:
batch_size = 32

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size)

## 1.5 Creating the Model

In [9]:
layers = []
layers

[]

In [10]:
layers = [
    nn.Flatten()
]
layers

[Flatten(start_dim=1, end_dim=-1)]

### 1.5.2 The Input Layer

In [11]:
input_size = 1 * 28 * 28

In [12]:
layers = [
    nn.Flatten(),
    nn.Linear(input_size, 512),  # Input
    nn.ReLU(),  # Activation for input
]
layers

[Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=784, out_features=512, bias=True),
 ReLU()]

### 1.5.3 The Hidden Layer

In [13]:
layers = [
    nn.Flatten(),
    nn.Linear(input_size, 512),  # Input
    nn.ReLU(),  # Activation for input
    nn.Linear(512, 512),  # Hidden
    nn.ReLU()  # Activation for hidden
]
layers

[Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=784, out_features=512, bias=True),
 ReLU(),
 Linear(in_features=512, out_features=512, bias=True),
 ReLU()]

In [14]:
n_classes = 10

layers = [
    nn.Flatten(),
    nn.Linear(input_size, 512),  # Input
    nn.ReLU(),  # Activation for input
    nn.Linear(512, 512),  # Hidden
    nn.ReLU(),  # Activation for hidden
    nn.Linear(512, n_classes)  # Output
]
layers

[Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=784, out_features=512, bias=True),
 ReLU(),
 Linear(in_features=512, out_features=512, bias=True),
 ReLU(),
 Linear(in_features=512, out_features=10, bias=True)]

### 1.5.5 Compiling the Model

In [15]:
model = nn.Sequential(*layers)
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=512, bias=True)
  (4): ReLU()
  (5): Linear(in_features=512, out_features=10, bias=True)
)

In [16]:
model.to(device)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=512, bias=True)
  (4): ReLU()
  (5): Linear(in_features=512, out_features=10, bias=True)
)

In [17]:
next(model.parameters()).device

device(type='cuda', index=0)

In [18]:
model = torch.compile(model)

## 1.6 Training the Model

In [19]:
loss_function = nn.CrossEntropyLoss()

In [20]:
optimizer = Adam(model.parameters())

### 1.6.2 Calculating Accuracy

In [21]:
train_N = len(train_loader.dataset)
valid_N = len(valid_loader.dataset)
train_N

60000

In [22]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

### 1.6.3 The Train Function

In [26]:
def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

### 1.6.4 The Validate Function

In [24]:
def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

### 1.6.5 The Training Loop

In [31]:
epochs = 5

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()

Epoch: 0


W1022 20:02:31.854000 134525777928832 torch/_dynamo/convert_frame.py:1009] WON'T CONVERT inner /usr/local/lib/python3.10/dist-packages/torch/_dynamo/external_utils.py line 36 
W1022 20:02:31.854000 134525777928832 torch/_dynamo/convert_frame.py:1009] due to: 
W1022 20:02:31.854000 134525777928832 torch/_dynamo/convert_frame.py:1009] Traceback (most recent call last):
W1022 20:02:31.854000 134525777928832 torch/_dynamo/convert_frame.py:1009]   File "/usr/local/lib/python3.10/dist-packages/torch/_dynamo/convert_frame.py", line 948, in __call__
W1022 20:02:31.854000 134525777928832 torch/_dynamo/convert_frame.py:1009]     result = self._inner_convert(
W1022 20:02:31.854000 134525777928832 torch/_dynamo/convert_frame.py:1009]   File "/usr/local/lib/python3.10/dist-packages/torch/_dynamo/convert_frame.py", line 472, in __call__
W1022 20:02:31.854000 134525777928832 torch/_dynamo/convert_frame.py:1009]     return _compile(
W1022 20:02:31.854000 134525777928832 torch/_dynamo/convert_frame.py:

Train - Loss: 375.9773 Accuracy: 0.9386
Valid - Loss: 27.8605 Accuracy: 0.9709
Epoch: 1
Train - Loss: 158.4767 Accuracy: 0.9740
Valid - Loss: 25.9105 Accuracy: 0.9742
Epoch: 2
Train - Loss: 106.5978 Accuracy: 0.9820
Valid - Loss: 27.0370 Accuracy: 0.9739
Epoch: 3
Train - Loss: 84.5658 Accuracy: 0.9859
Valid - Loss: 24.2619 Accuracy: 0.9766
Epoch: 4
Train - Loss: 65.3999 Accuracy: 0.9889
Valid - Loss: 23.5447 Accuracy: 0.9789


In [38]:
train_set[0][0].shape


torch.Size([1, 28, 28])

In [39]:
prediction = model(train_set[0][0].to(device))
prediction

tensor([[-23.7735, -10.0770, -12.4272,   6.9609, -24.3911,  14.9587, -16.4885,
         -14.1184, -11.9873,   0.2044]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [40]:
prediction.argmax(dim=1, keepdim=True)

tensor([[5]], device='cuda:0')

Did it get it right?

### 1.7.1 Clear the Memory

Before moving on, please execute the following cell to clear up the GPU memory. This is required to move on to the next notebook.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<center><a href="https://www.nvidia.com/dli"> <img src="https://github.com/sathyasheelans/NVIDIA_DL/blob/master/Nvidia%20-%20Intro%20to%20Deep%20Learning/Lesson_1/images/DLI_Header.png?raw=1" alt="Header" style="width: 400px;"/> </a></center>